In [40]:
import pygame
import numpy as np
import matplotlib.pyplot as plt
from beatmapparser import *
import beatmapparser
from curve import *
from slidercalc import *
import math

In [69]:
parser = beatmapparser.BeatmapParser()
parser.parseFile('leaf.osu')
beatmap_d = parser.build_beatmap()

In [70]:
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

def dist(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

def calc_fade_in(time, t1, t2, limit=1):
    # goes from 0 to limit
    return limit * (1 / (t2 - t1)) * (time - t1)

def calc_fade_out(time, t1, t2, limit=1):
    # goes from limit to 0
    return limit * (-1 / (t2 - t1)) * (time - t2)


In [71]:
def draw_point(surface, point, a):
    old_color = surface.get_at(point)
    new_r = int(old_color.r * (1 - a) + 255 * a)
    new_g = int(old_color.g * (1 - a) + 255 * a)
    new_b = int(old_color.b * (1 - a) + 255 * a)
    surface.set_at(point, (new_r, new_g, new_b))

def draw_filled_circle(surface, center, radius, a):
    tl_x, tl_y = max(0, center[0] - radius), max(0, center[1] - radius)
    br_x, br_y = min(tl_x + 2 * radius + 1, surface.get_width()), min(tl_y + 2 * radius + 1, surface.get_height())
    for i in range(tl_x, br_x):
        for j in range(tl_y, br_y):
            if dist(center, (i, j)) < radius:
                draw_point(surface, (i, j), a)

def draw_empty_circle(surface, center, radius, a, width=2):
    tl_x, tl_y = max(0, center[0] - radius - width // 2), max(0, center[1] - radius - width // 2)
    br_x, br_y = min(center[0] + radius + width // 2 + 1, surface.get_width()), min(center[1] + radius + width // 2 + 1, surface.get_height())
    for i in range(tl_x, br_x):
        for j in range(tl_y, br_y):
            if radius - width // 2 <= dist(center, (i, j)) < radius + width // 2:
                draw_point(surface, (i, j), a)

def should_draw_point(point, centers, radius):
    np.random.shuffle(centers)
    for center in centers:
        if dist(center, point) <= radius:
            return True
    return False
                
def draw_slider_body(surface, points, radius, a):
    centers = set()
    for t in np.arange(0, 1, 0.05):
        px = int(sum([points[i][0] * (1 - t) ** (len(points) - i - 1) * t ** (i) * math.comb(len(points) - 1, i) for i in range(len(points))]))
        py = int(sum([points[i][1] * (1 - t) ** (len(points) - i - 1) * t ** (i) * math.comb(len(points) - 1, i) for i in range(len(points))]))
        centers.add((px, py))
    centers = list(centers)
        
    l = max(0, min(centers, key=lambda x: x[0])[0] - radius)
    r = min(max(centers, key=lambda x: x[0])[0] + radius + 1, surface.get_width())
    b = max(0, min(centers, key=lambda x: x[1])[1] - radius)
    t = min(max(centers, key=lambda x: x[1])[1] + radius + 1, surface.get_height())
    
    for i in range(l, r):
        for j in range(b, t):
            if should_draw_point((i, j), centers, radius):
                draw_point(surface, (i, j), a)
                

In [72]:
def draw_cursor(surface, center, radius):
    tl_x, tl_y = max(0, center[0] - radius), max(0, center[1] - radius)
    br_x, br_y = min(tl_x + 2 * radius + 1, surface.get_width()), min(tl_y + 2 * radius + 1, surface.get_height())
    for i in range(tl_x, br_x):
        for j in range(tl_y, br_y):
            if dist(center, (i, j)) < radius:
                surface.set_at((i, j), (255, 255, 0))

In [73]:
class Circle:
    def __init__(self, center, startTime, radius=10, approachRate=1000):
        self.center = center
        self.radius = radius
        self.startTime = startTime
        self.approachRate = approachRate
        self.appear = startTime - approachRate
        self.disappear = startTime + approachRate // 4
        
        self.hit = False
    
    def draw(self, screen, time, stack_offset=0):
        if self.hit: return
        if not self.appear <= time <= self.disappear: return
        
        # get fade in/out opacity
        if time < self.startTime:
            opacity = calc_fade_in(time, self.appear, self.startTime)
        else:
            opacity = calc_fade_out(time, self.startTime, self.disappear)
        
        # handle offset
        center = self.center
        if stack_offset:
            center = (self.center[0] - 2 * stack_offset, self.center[1] - 2 * stack_offset)
        
        # draw hit circle
        draw_filled_circle(screen, center, self.radius, opacity)
    
    def draw_approach_circle(self, screen, time, stack_offset=0):
        if self.hit: return
        if not self.appear <= time <= self.startTime: return
        
        # get fade in opacity
        opacity = calc_fade_in(time, self.appear, self.startTime, limit=0.5)
        
        # handle offset
        center = self.center
        if stack_offset:
            center = (self.center[0] - 2 * stack_offset, self.center[1] - 2 * stack_offset)
        
        # draw approach circle
        radius = int(self.radius + 3 * (self.startTime / (self.startTime - self.appear) - (1 / (self.startTime - self.appear)) * time) * self.radius)
        draw_empty_circle(screen, center, radius, opacity)
                    

In [74]:
class Slider:
    def __init__(self, startTime, endTime, points, repeatCount, radius=10, approach=1000):
        self.startTime = startTime
        self.endTime = endTime
        self.appear = startTime - approach
        self.disappear = endTime + approach // 4
        self.points = points
        self.numPoints = len(points)
        self.repeatCount = repeatCount
        self.radius = radius
        self.center = points[0]
    
    def draw(self, screen, time, stack_offset=0):
        # get circle/slider opacity
        if time < self.startTime:
            circle_opacity = calc_fade_in(time, self.appear, self.startTime)
        elif time <= self.endTime:
            circle_opacity = 1
        else:
            circle_opacity = calc_fade_out(time, self.endTime, self.disappear)
        slider_opacity = circle_opacity * 0.2

        # draw slider body
        draw_slider_body(screen, self.points, self.radius, slider_opacity)
        
        # keep the circle in the start/ending position
        if time < self.startTime:
            draw_filled_circle(screen, self.points[0], self.radius, circle_opacity)
        if time > self.endTime:
            end_point = self.points[0] if self.repeatCount % 2 == 0 else self.points[-1]
            draw_filled_circle(screen, end_point, self.radius, circle_opacity)
        
        # draw sliding hit circle
        if self.startTime <= time <= self.endTime:
            one = (self.endTime - self.startTime) / self.repeatCount 
            rep_num = (time - self.startTime) // one
            time_within_rep = (time - self.startTime) % one
            t = time_within_rep / one
            if rep_num % 2 != 0:
                t = 1 - t
            cx = int(sum([self.points[i][0] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)]))
            cy = int(sum([self.points[i][1] * (1 - t) ** (self.numPoints - i - 1) * t ** (i) * math.comb(self.numPoints - 1, i) for i in range(self.numPoints)]))
            draw_filled_circle(screen, (cx, cy), self.radius, circle_opacity)

    def draw_approach_circle(self, screen, time, stack_offset=0):
        if not self.appear <= time <= self.startTime: return
        
        # get fade in opacity
        opacity = calc_fade_in(time, self.appear, self.startTime, limit=0.5)
        
        # draw approach circle
        radius = int(self.radius + 3 * (self.startTime / (self.startTime - self.appear) - (1 / (self.startTime - self.appear)) * time) * self.radius)
        draw_empty_circle(screen, self.points[0], radius, opacity)
                

In [75]:
class Stack:
    def __init__(self, hitObjects):
        self.hitObjects = sorted(hitObjects, key=lambda x: x.startTime, reverse=True)
        self.appear = hitObjects[0].appear
        self.startTime = hitObjects[0].startTime
        self.disappear = hitObjects[-1].disappear
        
    def draw(self, screen, time):
        if not self.hitObjects: return
        for i, obj in enumerate(self.hitObjects):
            if obj.appear <= time <= obj.disappear:
                obj.draw(screen, time, stack_offset=i)
    
    def draw_approach_circle(self, screen, time):
        if not self.hitObjects: return
        for i, obj in enumerate(self.hitObjects):
            if obj.appear <= time <= obj.disappear:
                obj.draw_approach_circle(screen, time, stack_offset=i)

In [76]:
class Frame:
    def __init__(self, hitObjects):
        self.hitObjects = sorted(hitObjects, key=lambda x: x.startTime, reverse=True)
    
    def draw(self, screen, time):
        if not self.hitObjects: return
        for obj in self.hitObjects:
            obj.draw(screen, time)
        for obj in self.hitObjects: # draw approach circles on top of everything
            obj.draw_approach_circle(screen, time)

In [77]:
def convert(hitObject):
    if hitObject['object_name'] == 'circle':
        x, y = hitObject['position']
        scaled_center = ((x * 192 // 512, y * 192 // 512))
        return Circle(scaled_center, hitObject['startTime'])
    elif hitObject['object_name'] == 'slider':
        scaled_points = []
        for x, y in hitObject['points']:
            scaled_points.append((x * 192 // 512, y * 192 // 512))
        return Slider(hitObject['startTime'], hitObject['end_time'], scaled_points, hitObject['repeatCount'])

def same_pos(unscaled, scaled):
    return unscaled[0] * 192 // 512 == scaled[0] and unscaled[1] * 192 // 512 == scaled[1]
    
def FormatParsedBeatmap(beatmap_d: dict):
    hitObjects = []
    stack = []
    count = 0
    for hitObject in beatmap_d['hitObjects']:
        
        if hitObject['object_name'] not in ['circle', 'slider']: continue
        
        if not stack and hitObjects and same_pos(hitObject['position'], hitObjects[-1].center):
            stack.append(hitObjects[-1])
            stack.append(convert(hitObject))
            hitObjects.pop()
            continue
        elif stack and same_pos(hitObject['position'], stack[-1].center):
            stack.append(convert(hitObject))
            continue
        if stack: 
            hitObjects.append(Stack(stack))
            stack = []
        count += 1
        hitObjects.append(convert(hitObject))
    return hitObjects


In [78]:
hitObjects = FormatParsedBeatmap(beatmap_d)

In [ ]:
import sys, pygame
from pygame.locals import *
pygame.quit()
pygame.init()

size = width, height = 192, 144
clock = pygame.time.Clock()

screen = pygame.display.set_mode(size, DOUBLEBUF, 16)
screen.fill(BLACK)

ms = pygame.Surface((10, 10))
draw_cursor(ms, (5, 5), 5)
cursor = pygame.cursors.Cursor((5, 5), ms)
pygame.mouse.set_cursor(cursor)

run = True
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False
            
    screen.fill(BLACK)
    
    objs = []
    for obj in hitObjects:
        if obj.appear < pygame.time.get_ticks() < obj.disappear:
            objs.append(obj)

    frame = Frame(objs)
    frame.draw(screen, pygame.time.get_ticks())
    
    pygame.display.flip()
    pygame.display.update()
    
    if pygame.time.get_ticks() > hitObjects[-1].disappear + 2000:
        run = False
    clock.tick(60)
